In [ ]:
!pip install adversarial robustness toolbox
!pip install foolbox
!pip install umap-learn
!pip install multiprocess

In [ ]:
import pandas as np
import numpy as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import umap
from umap import UMAP
from tensorflow.keras.models import load_model
from art.estimators.classification import TensorFlowV2Classifier
from tensorflow.keras.datasets import mnist
#from art.attacks.evasion import FastGradientMethod, CarliniLInfMethod
from art.attacks.evasion import DeepFool

In [ ]:
# Load and preprocess MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()


# Subset the test data to 1000 samples
x_test = x_test[:7000]
y_test = y_test[:7000]

# Flatten and normalize
x_test_flat = x_test.reshape((x_test.shape[0], -1)) / 255.0

# Create a copy of the original data
legitimate_x = np.copy(x_test_flat)
legitimate_y = np.copy(y_test)

In [ ]:
# Fit UMAP for dimensionality reduction
umap_2d = UMAP(n_neighbors=15, n_components=2, min_dist=0.1, random_state=42)
umap_3d = UMAP(n_neighbors=15, n_components=3, min_dist=0.1, random_state=42)
umap_5d = UMAP(n_neighbors=15, n_components=5, min_dist=0.1, random_state=42)
umap_10d = UMAP(n_neighbors=15, n_components=10, min_dist=0.1, random_state=42)

In [ ]:
# Fit UMAP on legitimate data
umap_2d_legitimate = umap_2d.fit_transform(legitimate_x)
umap_3d_legitimate = umap_3d.fit_transform(legitimate_x)
umap_5d_legitimate = umap_5d.fit_transform(legitimate_x)
umap_10d_legitimate = umap_10d.fit_transform(legitimate_x)

In [ ]:
# Load pre-trained CNN model
model = load_model("mnist_CNN_model.h5")

# Create ART classifier wrapper (no input_shape)
loss_object = tf.keras.losses.CategoricalCrossentropy()
deepfool_classifier = TensorFlowV2Classifier(
    model=model,
    loss_object=loss_object,
    nb_classes=10,
    input_shape=(28, 28, 1),
    clip_values=(0.0, 1.0)
)

In [ ]:
# Reshape legitimate inputs for CNN
legitimate_x = legitimate_x.reshape(-1, 28, 28, 1)

# 10. Create DeepFool attack with batch_size
deepfool_attack = DeepFool(
    classifier=deepfool_classifier,
    max_iter=100,
    epsilon=1e-6,
    nb_grads=20,
    batch_size=64,
    verbose=True
)

# Generate adversarial examples
x_test_adv_deepfool = deepfool_attack.generate(x=legitimate_x)


In [ ]:
# sanity check on clean images

# Predict the labels before and after the attack
preds_legit = np.argmax(model.predict(legitimate_x), axis=1)
preds_adv = np.argmax(model.predict(x_test_adv_deepfool), axis=1)

# Compute the number of successful attacks
successful_attacks = np.sum(preds_legit != preds_adv)
total_samples = len(legitimate_x)
success_rate = successful_attacks / total_samples

print(f"DeepFool Success Rate: {success_rate:.2%}")

In [ ]:
# Flatten for UMAP
x_adv_deepfool = x_test_adv_deepfool.reshape((x_test_adv_deepfool.shape[0], -1))

# UMAP projection of adversarial examples
adv_2d_deepfool = umap_2d.fit_transform(x_adv_deepfool)
adv_3d_deepfool = umap_3d.fit_transform(x_adv_deepfool)
adv_5d_deepfool = umap_5d.fit_transform(x_adv_deepfool)
adv_10d_deepfool = umap_10d.fit_transform(x_adv_deepfool)

In [ ]:
# Create UMAP transformers for 2D
column_names = [f"umap{i}" for i in range(2)]
data_2d_legitimate = pd.DataFrame(umap_2d_legitimate, columns=column_names)
data_2d_deepfool = pd.DataFrame(adv_2d_deepfool, columns=column_names)

# Create UMAP transformers for 3D
column_names = [f"umap{i}" for i in range(3)]
data_3d_legitimate = pd.DataFrame(umap_3d_legitimate, columns=column_names)
data_3d_deepfool = pd.DataFrame(adv_3d_deepfool, columns=column_names)

# Create UMAP transformers for 5D
column_names = [f"umap{i}" for i in range(5)]
data_5d_legitimate = pd.DataFrame(umap_5d_legitimate, columns=column_names)
data_5d_deepfool = pd.DataFrame(adv_5d_deepfool, columns=column_names)

# Create UMAP transformers for 10D
column_names = [f"umap{i}" for i in range(10)]
data_10d_legitimate = pd.DataFrame(umap_10d_legitimate, columns=column_names)
data_10d_deepfool = pd.DataFrame(adv_10d_deepfool, columns=column_names)

In [ ]:
# Add labels for the features
data_2d_legitimate["attack"] = 0  # Legitimate samples labeled as 0
data_2d_deepfool["attack"] = 1  # FGSM attack samples labeled as 1

data_3d_legitimate["attack"] = 0  
data_3d_deepfool["attack"] = 1  

data_5d_legitimate["attack"] = 0  
data_5d_deepfool["attack"] = 1  

data_10d_legitimate["attack"] = 0  
data_10d_deepfool["attack"] = 1  


# Combine the dataframes
UMAP2D_data = pd.concat([data_2d_legitimate, data_2d_deepfool], axis=0)
UMAP3D_data = pd.concat([data_3d_legitimate, data_3d_deepfool], axis=0)
UMAP5D_data = pd.concat([data_5d_legitimate, data_5d_deepfool], axis=0)
UMAP10D_data = pd.concat([data_10d_legitimate, data_10d_deepfool], axis=0)


# Save the dataframe to a CSV file
UMAP2D_data.to_csv("Umap_data_2D_deepfool_7000.csv", index=False)
UMAP3D_data.to_csv("Umap_data_3D_deepfoo1_7000.csv", index=False)
UMAP5D_data.to_csv("Umap_data_5D_deepfool_7000.csv", index=False)
UMAP10D_data.to_csv("Umap_data_10D_deepfool_7000.csv", index=False)